In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import time
import torch
from torch import logsumexp
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions.normal import Normal
import numpy as np

In [ ]:
iterations = 6000
num_samples = 100
p_mu = 0.0

lr = 5*1e-3
log_Z = np.log(np.sqrt((2*np.pi)))

In [ ]:
def rws(num_samples, q_mu, q_sigma, lr):
    EUBO = []
    ELBO = []
    Mu = []
    Grad_mu = []
    SNR_mu = []
    for i in range(iterations):
        proposal = Normal(q_mu, q_sigma)
        xs = proposal.sample((num_samples,))
        log_gammas = (-1.0 / 2.0) * ((xs - p_mu) ** 2)
        log_q = proposal.log_prob(xs)

        log_weights = log_gammas - log_q
        weights = torch.exp(log_weights - logsumexp(log_weights, dim=0)).detach()
        eubo = torch.mul(weights, log_weights).sum()
        elbo = log_weights.mean()
        gradient = torch.autograd.grad(eubo, q_mu)
        q_mu = q_mu - lr * gradient[0]
        EUBO.append(eubo.item())
        ELBO.append(elbo.item())
        Mu.append(q_mu.item())
        SNR_mu.append()
        Grad_mu.append(gradient[0].item())
#         AdamGrad = smooth_exponential(Grad, adam_b1) / (smooth_exponential(Grad, adam_b2) ** 0.5) 
    return EUBO, ELBO, Mu, Grad_mu, SNR_mu

In [ ]:
def train(mu, init_sigmas, num_samples, lr):
    num_sigmas = init_sigmas.shape[0]
    EUBOs = []
    ELBOs = []
    Grad_mus = []
    Mus = []
    SNR_mus = []
    for j in range(num_sigmas):
        time_start = time.time()
        q_mu = torch.tensor([1.0], requires_grad=True) * mu
        q_sigma = torch.tensor([1.0]) * init_sigmas[j]
        EUBO, ELBO, Mu, Grad_mu, SNR_mu = rws(num_samples, q_mu, q_sigma, lr)
        EUBOs.append(EUBO)
        ELBOs.append(ELBO)
        Mus.append(Mu)
        Grad_mus.append(Grad_mu)
        SNR_mus.append(SNR_mu)
        time_end = time.time()
        print('init_mu=%d, init_sigma=%d, samples : %d (%ds)' % (init_mus[i], init_sigmas[j], num_samples, time_end - time_start))    
    return EUBOs, ELBOs, Mus, Grad_mus, SNR_mus

In [ ]:
def plot_results(EUBOs, ELBOs, Mus, Grad_mus, SNR_mus, mu, init_sigmas, num_samples):
    num_sigmas = init_sigmas.shape[0]
    fig = plt.figure(figsize=(20, 15))
    for j in range(num_sigmas):
        fig.tight_layout()
        ax1 = fig.add_subplot(num_sigmas, 3, j*3 + 1)
        ax2 = fig.add_subplot(num_sigmas, 3, j*3 + 2)
        ax3 = fig.add_subplot(num_sigmas, 3, j*3 + 3)
        ax1.plot(EUBOs[j], 'r', label='EUBOs')
        ax1.plot(ELBOs[j], 'b', label='ELBOs')
        ax1.plot(np.ones(iterations) * log_Z, 'k', label='log_Z')
        ax1.set_ylim([-50, 20])
        ax1.tick_params(labelsize=14)
        ax2.plot(Mus[j], 'm', label='mu')
        ax2.tick_params(labelsize=14)
        ax3.plot(Grad_mus[j], 'g', label='grad mu')
        ax3.tick_params(labelsize=14)
        ax3.set_ylim([-8, 3])        

        ax1.set_title('mu=%d, sigma=%d, samples=%d' % (mu, init_sigmas[j], num_samples), fontsize=11)
        if j == 0:
            ax1.legend()
            ax2.legend()
            ax3.legend()

    plt.savefig('univariate_gaussian_rws_onlylearn_mu=%d_samples=%d_iters=%d.svg' % (mu, num_samples, iterations))

In [ ]:
init_mus = np.array([6, 8, 10])
init_sigmas = np.array([1.0, 2.0, 4.0, 6.0])
num_mus = init_mus.shape[0]

for i in range(num_mus):
    EUBOs, ELBOs, Mus, Grad_mus, SNR_mus = train(init_mus[i], init_sigmas, num_samples, lr)
    plot_results(EUBOs, ELBOs, Mus, Grad_mus, SNR_mus, init_mus[i], init_sigmas, num_samples)